In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from cuda_functional import SRU, SRUCell

import re
import tqdm
import jieba
import random

# 1.数据处理部分

In [2]:
USE_CUDA = True
path = './data/cmn-eng/'

SOS_token = 0
EOS_token = 1

In [3]:
def isChinese(sen):
    zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
    return zhPattern.search(sen)
# 简化句子 便于处理
def normalize_string(s):
    s = re.sub(r"[!！？.()（）""?。“”，,']", r" ", s)
    return s

In [4]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # Count SOS and EOS
      
    def index_words(self, sentence):
        sen_list = []
        if isChinese(sentence):
            sen_list = jieba.cut(sentence)
        else:
            sen_list = sentence.split(' ')
            
        for word in sen_list:
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
def read_sen(path, lang1, lang2, reverse=False):
    with open(path + '{}-{}.txt'.format(lang1, lang2)) as f:
        lines = f.readlines()
        pairs = []
        for line in lines:
            line = line.strip()
            if reverse:
                line = line.split('\t')
                line.reverse()
                line = "\t".join(line)
                
            pair = [normalize_string(sen) for sen in line.split('\t')]
            pairs.append(pair)
        
        if reverse:
            input_lang = Lang(lang2)
            output_lang = Lang(lang1) 
        else:
            input_lang = Lang(lang1)            
            output_lang = Lang(lang2)   
            
    return input_lang, output_lang, pairs

In [6]:
def data_preprocess(path, lang1, lang2, reverse=False):
    print("Read lines......")
    input_lang, output_lang, pairs = read_sen(path, lang1, lang2, reverse)
    print("Trimmed  to {} sentence pairs".format(len(pairs)))
    
    print("Indexing words......")
    for pair in pairs:
        input_lang.index_words(pair[0])
        output_lang.index_words(pair[1])
    
    return input_lang, output_lang, pairs
    
input_lang, output_lang, pairs = data_preprocess(path, 'eng', 'cmn')
for i in range(5):
    print(random.choice(pairs))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


Read lines......
Trimmed  to 19056 sentence pairs
Indexing words......


Loading model cost 0.696 seconds.
Prefix dict has been built succesfully.


['I found a beautiful shell on the shore ', '我在海滩上发现了一个漂亮的贝壳 ']
['She is going to wash the bike this afternoon ', '她下午会洗自行车 ']
['It s been a long war ', '这是场长久的战争 ']
['There was a heavy rain last night ', '昨晚下大雨 ']
['I m counting on your help ', '我指望你的帮助 ']


# 2.pytorch 搭建模型

## 2.1.数据部分

In [8]:
#sen's index
def ixs_from_sen(lang, sen):
    if isChinese(lang):
        sen = jieba.cut('')
    else:
        sen = sen.split(' ')
        
    return [lang.word2index[word] for word in sen]

def var_from_sen(lang, sen):
    ixs = ixs_from_sen(lang, sen)
    ixs.append(EOS_token)
    var = Variable(torch.LongTensor(ixs).view(-1, 1))
    if USE_CUDA: 
        var = var.cuda()
    
    return var
    

def var_from_pair(pair):
    input_variable = var_from_sen(in)
    input_variable = 

    return (input_variable, output_variable)

## 2.2.模型搭建

## 2.3.训练